# Filtrado Cruzado entre Elecciones

Este notebook implementa el análisis de **filtrado cruzado** entre Generales y Ballotage.

## Concepto:

Actualmente, cada elección usa sus propios ítems significativos:
- Generales → filtra con ítems significativos de Generales
- Ballotage → filtra con ítems significativos de Ballotage

**NUEVO: Filtrado Cruzado**
- Ballotage filtrado con ítems significativos de **Generales**
- Generales filtrado con ítems significativos de **Ballotage**

## Objetivo:

1. Validación cruzada de hallazgos
2. Robustez de ítems significativos
3. Comparar consistencia entre elecciones

## Proceso:

1. Identificar ítems significativos (p<0.05, Kruskal-Wallis) en cada elección
2. Crear variables filtradas cruzadas
3. Agregar a df_Elecciones
4. Análisis estadístico comparativo

In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from scipy.stats import kruskal, wilcoxon
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

✓ Librerías cargadas exitosamente


## 1. Cargar Datos

In [2]:
# Rutas
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

✓ Datos cargados:
  - Generales: 2786 registros
  - Ballotage: 1254 registros


## 2. Definir Listas de Ítems

In [3]:
# ÍTEMS PROGRESISTAS: 5, 6, 9, 11, 16, 20, 24, 25, 27, 28
# ÍTEMS CONSERVADORES: 3, 4, 7, 8, 10, 19, 22, 23, 29, 30

# Cambio de Opinión
CO_Progresistas_Izquierda = [
    'CO_Item_5_Izq', 'CO_Item_6_Izq', 'CO_Item_9_Izq', 'CO_Item_11_Izq', 'CO_Item_16_Izq',
    'CO_Item_20_Izq', 'CO_Item_24_Izq', 'CO_Item_25_Izq', 'CO_Item_27_Izq', 'CO_Item_28_Izq'
]

CO_Progresistas_Derecha = [
    'CO_Item_5_Der', 'CO_Item_6_Der', 'CO_Item_9_Der', 'CO_Item_11_Der', 'CO_Item_16_Der',
    'CO_Item_20_Der', 'CO_Item_24_Der', 'CO_Item_25_Der', 'CO_Item_27_Der', 'CO_Item_28_Der'
]

CO_Conservadores_Izquierda = [
    'CO_Item_3_Izq', 'CO_Item_4_Izq', 'CO_Item_7_Izq', 'CO_Item_8_Izq', 'CO_Item_10_Izq',
    'CO_Item_19_Izq', 'CO_Item_22_Izq', 'CO_Item_23_Izq', 'CO_Item_29_Izq', 'CO_Item_30_Izq'
]

CO_Conservadores_Derecha = [
    'CO_Item_3_Der', 'CO_Item_4_Der', 'CO_Item_7_Der', 'CO_Item_8_Der', 'CO_Item_10_Der',
    'CO_Item_19_Der', 'CO_Item_22_Der', 'CO_Item_23_Der', 'CO_Item_29_Der', 'CO_Item_30_Der'
]

# Cambio de Tiempo
CT_Progresistas_Izquierda = [
    'CT_Item_5_Izq', 'CT_Item_6_Izq', 'CT_Item_9_Izq', 'CT_Item_11_Izq', 'CT_Item_16_Izq',
    'CT_Item_20_Izq', 'CT_Item_24_Izq', 'CT_Item_25_Izq', 'CT_Item_27_Izq', 'CT_Item_28_Izq'
]

CT_Progresistas_Derecha = [
    'CT_Item_5_Der', 'CT_Item_6_Der', 'CT_Item_9_Der', 'CT_Item_11_Der', 'CT_Item_16_Der',
    'CT_Item_20_Der', 'CT_Item_24_Der', 'CT_Item_25_Der', 'CT_Item_27_Der', 'CT_Item_28_Der'
]

CT_Conservadores_Izquierda = [
    'CT_Item_3_Izq', 'CT_Item_4_Izq', 'CT_Item_7_Izq', 'CT_Item_8_Izq', 'CT_Item_10_Izq',
    'CT_Item_19_Izq', 'CT_Item_22_Izq', 'CT_Item_23_Izq', 'CT_Item_29_Izq', 'CT_Item_30_Izq'
]

CT_Conservadores_Derecha = [
    'CT_Item_3_Der', 'CT_Item_4_Der', 'CT_Item_7_Der', 'CT_Item_8_Der', 'CT_Item_10_Der',
    'CT_Item_19_Der', 'CT_Item_22_Der', 'CT_Item_23_Der', 'CT_Item_29_Der', 'CT_Item_30_Der'
]

# Todas las columnas CO y CT
Columnas_CO = (CO_Progresistas_Izquierda + CO_Progresistas_Derecha + 
               CO_Conservadores_Izquierda + CO_Conservadores_Derecha)

Columnas_CT = (CT_Progresistas_Izquierda + CT_Progresistas_Derecha + 
               CT_Conservadores_Izquierda + CT_Conservadores_Derecha)

print(f"✓ Definidas {len(Columnas_CO)} columnas CO y {len(Columnas_CT)} columnas CT")

✓ Definidas 40 columnas CO y 40 columnas CT


## 3. Identificar Ítems Significativos en Cada Elección

Aplicar Kruskal-Wallis para cada ítem individual y determinar cuáles son significativos (p < 0.05).

In [4]:
# Categorías válidas
Categorias_Validas = [
    'Left_Wing', 'Progressivism', 'Centre',
    'Moderate_Right_A', 'Moderate_Right_B', 'Right_Wing_Libertarian'
]

# Diccionario para almacenar ítems significativos
Items_Significativos = {
    'Generales': {'CO': [], 'CT': []},
    'Ballotage': {'CO': [], 'CT': []}
}

print("="*70)
print("IDENTIFICACIÓN DE ÍTEMS SIGNIFICATIVOS (Kruskal-Wallis, p < 0.05)")
print("="*70)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n📊 {Nombre_df}:")
    print("-"*70)
    
    # Filtrar categorías válidas
    df_filtrado = df[df['Categoria_PASO_2023'].isin(Categorias_Validas)]
    
    # ANÁLISIS CO
    print("\n  🔍 Analizando ítems CO...")
    for columna in Columnas_CO:
        if columna in df_filtrado.columns:
            # Agrupar por categoría
            grupos = [
                df_filtrado[df_filtrado['Categoria_PASO_2023'] == cat][columna].dropna()
                for cat in Categorias_Validas
            ]
            
            # Verificar que todos los grupos tengan datos
            if all(len(grupo) > 1 for grupo in grupos):
                estadistico, p_valor = kruskal(*grupos)
                
                if p_valor < 0.05:
                    Items_Significativos[Nombre_df]['CO'].append(columna)
    
    print(f"    ✓ Ítems CO significativos: {len(Items_Significativos[Nombre_df]['CO'])}")
    
    # ANÁLISIS CT
    print("\n  🔍 Analizando ítems CT...")
    for columna in Columnas_CT:
        if columna in df_filtrado.columns:
            # Agrupar por categoría
            grupos = [
                df_filtrado[df_filtrado['Categoria_PASO_2023'] == cat][columna].dropna()
                for cat in Categorias_Validas
            ]
            
            # Verificar que todos los grupos tengan datos
            if all(len(grupo) > 1 for grupo in grupos):
                estadistico, p_valor = kruskal(*grupos)
                
                if p_valor < 0.05:
                    Items_Significativos[Nombre_df]['CT'].append(columna)
    
    print(f"    ✓ Ítems CT significativos: {len(Items_Significativos[Nombre_df]['CT'])}")

print("\n" + "="*70)
print("RESUMEN DE ÍTEMS SIGNIFICATIVOS")
print("="*70)
print(f"\nGenerales:")
print(f"  - CO: {Items_Significativos['Generales']['CO']}")
print(f"  - CT: {Items_Significativos['Generales']['CT']}")
print(f"\nBallotage:")
print(f"  - CO: {Items_Significativos['Ballotage']['CO']}")
print(f"  - CT: {Items_Significativos['Ballotage']['CT']}")
print("\n" + "="*70)

IDENTIFICACIÓN DE ÍTEMS SIGNIFICATIVOS (Kruskal-Wallis, p < 0.05)

📊 Generales:
----------------------------------------------------------------------

  🔍 Analizando ítems CO...
    ✓ Ítems CO significativos: 11

  🔍 Analizando ítems CT...
    ✓ Ítems CT significativos: 14

📊 Ballotage:
----------------------------------------------------------------------

  🔍 Analizando ítems CO...
    ✓ Ítems CO significativos: 9

  🔍 Analizando ítems CT...
    ✓ Ítems CT significativos: 5

RESUMEN DE ÍTEMS SIGNIFICATIVOS

Generales:
  - CO: ['CO_Item_5_Izq', 'CO_Item_9_Izq', 'CO_Item_24_Izq', 'CO_Item_25_Izq', 'CO_Item_3_Izq', 'CO_Item_7_Izq', 'CO_Item_8_Izq', 'CO_Item_10_Izq', 'CO_Item_30_Izq', 'CO_Item_10_Der', 'CO_Item_30_Der']
  - CT: ['CT_Item_16_Izq', 'CT_Item_9_Der', 'CT_Item_16_Der', 'CT_Item_25_Der', 'CT_Item_3_Izq', 'CT_Item_4_Izq', 'CT_Item_7_Izq', 'CT_Item_22_Izq', 'CT_Item_30_Izq', 'CT_Item_3_Der', 'CT_Item_4_Der', 'CT_Item_7_Der', 'CT_Item_10_Der', 'CT_Item_23_Der']

Ballotage:
  - C

## 4. Crear Variables Filtradas Cruzadas

Crear las nuevas variables usando el filtro "cruzado":
- En Ballotage: usar ítems significativos de Generales
- En Generales: usar ítems significativos de Ballotage

In [5]:
print("="*70)
print("CREANDO VARIABLES FILTRADAS CRUZADAS")
print("="*70)

def Filtrar_Items_Por_Categoria(items_significativos, lista_categoria):
    """Filtra los ítems significativos que pertenecen a una categoría específica."""
    return [item for item in items_significativos if item in lista_categoria]

# BALLOTAGE con filtro de GENERALES
print("\n📊 Ballotage (usando ítems significativos de Generales):")
print("-"*70)

# CO en Ballotage con filtro de Generales
Items_CO_Pro_Izq_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CO'], CO_Progresistas_Izquierda
)
Items_CO_Pro_Der_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CO'], CO_Progresistas_Derecha
)
Items_CO_Con_Izq_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CO'], CO_Conservadores_Izquierda
)
Items_CO_Con_Der_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CO'], CO_Conservadores_Derecha
)

df_Ballotage['CO_Filt_Pro_Izq_SegunGen'] = (
    df_Ballotage[Items_CO_Pro_Izq_Gen].sum(axis=1) if Items_CO_Pro_Izq_Gen else 0
)
df_Ballotage['CO_Filt_Pro_Der_SegunGen'] = (
    df_Ballotage[Items_CO_Pro_Der_Gen].sum(axis=1) if Items_CO_Pro_Der_Gen else 0
)
df_Ballotage['CO_Filt_Con_Izq_SegunGen'] = (
    df_Ballotage[Items_CO_Con_Izq_Gen].sum(axis=1) if Items_CO_Con_Izq_Gen else 0
)
df_Ballotage['CO_Filt_Con_Der_SegunGen'] = (
    df_Ballotage[Items_CO_Con_Der_Gen].sum(axis=1) if Items_CO_Con_Der_Gen else 0
)

print(f"  CO_Filt_Pro_Izq_SegunGen: {len(Items_CO_Pro_Izq_Gen)} ítems")
print(f"  CO_Filt_Pro_Der_SegunGen: {len(Items_CO_Pro_Der_Gen)} ítems")
print(f"  CO_Filt_Con_Izq_SegunGen: {len(Items_CO_Con_Izq_Gen)} ítems")
print(f"  CO_Filt_Con_Der_SegunGen: {len(Items_CO_Con_Der_Gen)} ítems")

# CT en Ballotage con filtro de Generales
Items_CT_Pro_Izq_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CT'], CT_Progresistas_Izquierda
)
Items_CT_Pro_Der_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CT'], CT_Progresistas_Derecha
)
Items_CT_Con_Izq_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CT'], CT_Conservadores_Izquierda
)
Items_CT_Con_Der_Gen = Filtrar_Items_Por_Categoria(
    Items_Significativos['Generales']['CT'], CT_Conservadores_Derecha
)

df_Ballotage['CT_Filt_Pro_Izq_SegunGen'] = (
    df_Ballotage[Items_CT_Pro_Izq_Gen].sum(axis=1) if Items_CT_Pro_Izq_Gen else 0
)
df_Ballotage['CT_Filt_Pro_Der_SegunGen'] = (
    df_Ballotage[Items_CT_Pro_Der_Gen].sum(axis=1) if Items_CT_Pro_Der_Gen else 0
)
df_Ballotage['CT_Filt_Con_Izq_SegunGen'] = (
    df_Ballotage[Items_CT_Con_Izq_Gen].sum(axis=1) if Items_CT_Con_Izq_Gen else 0
)
df_Ballotage['CT_Filt_Con_Der_SegunGen'] = (
    df_Ballotage[Items_CT_Con_Der_Gen].sum(axis=1) if Items_CT_Con_Der_Gen else 0
)

print(f"\n  CT_Filt_Pro_Izq_SegunGen: {len(Items_CT_Pro_Izq_Gen)} ítems")
print(f"  CT_Filt_Pro_Der_SegunGen: {len(Items_CT_Pro_Der_Gen)} ítems")
print(f"  CT_Filt_Con_Izq_SegunGen: {len(Items_CT_Con_Izq_Gen)} ítems")
print(f"  CT_Filt_Con_Der_SegunGen: {len(Items_CT_Con_Der_Gen)} ítems")

# GENERALES con filtro de BALLOTAGE
print("\n📊 Generales (usando ítems significativos de Ballotage):")
print("-"*70)

# CO en Generales con filtro de Ballotage
Items_CO_Pro_Izq_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CO'], CO_Progresistas_Izquierda
)
Items_CO_Pro_Der_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CO'], CO_Progresistas_Derecha
)
Items_CO_Con_Izq_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CO'], CO_Conservadores_Izquierda
)
Items_CO_Con_Der_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CO'], CO_Conservadores_Derecha
)

df_Generales['CO_Filt_Pro_Izq_SegunBal'] = (
    df_Generales[Items_CO_Pro_Izq_Bal].sum(axis=1) if Items_CO_Pro_Izq_Bal else 0
)
df_Generales['CO_Filt_Pro_Der_SegunBal'] = (
    df_Generales[Items_CO_Pro_Der_Bal].sum(axis=1) if Items_CO_Pro_Der_Bal else 0
)
df_Generales['CO_Filt_Con_Izq_SegunBal'] = (
    df_Generales[Items_CO_Con_Izq_Bal].sum(axis=1) if Items_CO_Con_Izq_Bal else 0
)
df_Generales['CO_Filt_Con_Der_SegunBal'] = (
    df_Generales[Items_CO_Con_Der_Bal].sum(axis=1) if Items_CO_Con_Der_Bal else 0
)

print(f"  CO_Filt_Pro_Izq_SegunBal: {len(Items_CO_Pro_Izq_Bal)} ítems")
print(f"  CO_Filt_Pro_Der_SegunBal: {len(Items_CO_Pro_Der_Bal)} ítems")
print(f"  CO_Filt_Con_Izq_SegunBal: {len(Items_CO_Con_Izq_Bal)} ítems")
print(f"  CO_Filt_Con_Der_SegunBal: {len(Items_CO_Con_Der_Bal)} ítems")

# CT en Generales con filtro de Ballotage
Items_CT_Pro_Izq_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CT'], CT_Progresistas_Izquierda
)
Items_CT_Pro_Der_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CT'], CT_Progresistas_Derecha
)
Items_CT_Con_Izq_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CT'], CT_Conservadores_Izquierda
)
Items_CT_Con_Der_Bal = Filtrar_Items_Por_Categoria(
    Items_Significativos['Ballotage']['CT'], CT_Conservadores_Derecha
)

df_Generales['CT_Filt_Pro_Izq_SegunBal'] = (
    df_Generales[Items_CT_Pro_Izq_Bal].sum(axis=1) if Items_CT_Pro_Izq_Bal else 0
)
df_Generales['CT_Filt_Pro_Der_SegunBal'] = (
    df_Generales[Items_CT_Pro_Der_Bal].sum(axis=1) if Items_CT_Pro_Der_Bal else 0
)
df_Generales['CT_Filt_Con_Izq_SegunBal'] = (
    df_Generales[Items_CT_Con_Izq_Bal].sum(axis=1) if Items_CT_Con_Izq_Bal else 0
)
df_Generales['CT_Filt_Con_Der_SegunBal'] = (
    df_Generales[Items_CT_Con_Der_Bal].sum(axis=1) if Items_CT_Con_Der_Bal else 0
)

print(f"\n  CT_Filt_Pro_Izq_SegunBal: {len(Items_CT_Pro_Izq_Bal)} ítems")
print(f"  CT_Filt_Pro_Der_SegunBal: {len(Items_CT_Pro_Der_Bal)} ítems")
print(f"  CT_Filt_Con_Izq_SegunBal: {len(Items_CT_Con_Izq_Bal)} ítems")
print(f"  CT_Filt_Con_Der_SegunBal: {len(Items_CT_Con_Der_Bal)} ítems")

print("\n" + "="*70)
print("✅ VARIABLES FILTRADAS CRUZADAS CREADAS EXITOSAMENTE")
print("="*70)

CREANDO VARIABLES FILTRADAS CRUZADAS

📊 Ballotage (usando ítems significativos de Generales):
----------------------------------------------------------------------
  CO_Filt_Pro_Izq_SegunGen: 4 ítems
  CO_Filt_Pro_Der_SegunGen: 0 ítems
  CO_Filt_Con_Izq_SegunGen: 5 ítems
  CO_Filt_Con_Der_SegunGen: 2 ítems

  CT_Filt_Pro_Izq_SegunGen: 1 ítems
  CT_Filt_Pro_Der_SegunGen: 3 ítems
  CT_Filt_Con_Izq_SegunGen: 5 ítems
  CT_Filt_Con_Der_SegunGen: 5 ítems

📊 Generales (usando ítems significativos de Ballotage):
----------------------------------------------------------------------
  CO_Filt_Pro_Izq_SegunBal: 2 ítems
  CO_Filt_Pro_Der_SegunBal: 2 ítems
  CO_Filt_Con_Izq_SegunBal: 3 ítems
  CO_Filt_Con_Der_SegunBal: 2 ítems

  CT_Filt_Pro_Izq_SegunBal: 0 ítems
  CT_Filt_Pro_Der_SegunBal: 0 ítems
  CT_Filt_Con_Izq_SegunBal: 2 ítems
  CT_Filt_Con_Der_SegunBal: 3 ítems

✅ VARIABLES FILTRADAS CRUZADAS CREADAS EXITOSAMENTE


## 5. Cargar df_Elecciones y Agregar Nuevas Variables

In [6]:
# Cargar df_Elecciones
Ruta_Procesados = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Excel_Elecciones = os.path.join(Ruta_Procesados, 'df_Elecciones.xlsx')

if os.path.exists(Excel_Elecciones):
    df_Elecciones = pd.read_excel(Excel_Elecciones)
    print(f"✓ df_Elecciones cargado: {len(df_Elecciones)} registros")
else:
    print("⚠️ df_Elecciones no existe, creando desde cero...")
    # Crear df_Elecciones concatenando Generales y Ballotage
    df_Gen_Copy = df_Generales[['ID']].copy()
    df_Gen_Copy['Eleccion'] = 'Generales'
    
    df_Bal_Copy = df_Ballotage[['ID']].copy()
    df_Bal_Copy['Eleccion'] = 'Ballotage'
    
    df_Elecciones = pd.concat([df_Gen_Copy, df_Bal_Copy], ignore_index=True)
    print(f"✓ df_Elecciones creado: {len(df_Elecciones)} registros")

✓ df_Elecciones cargado: 2786 registros


In [7]:
# Preparar datos para merge
Variables_Cruzadas = [
    'CO_Filt_Pro_Izq_SegunGen', 'CO_Filt_Pro_Der_SegunGen',
    'CO_Filt_Con_Izq_SegunGen', 'CO_Filt_Con_Der_SegunGen',
    'CT_Filt_Pro_Izq_SegunGen', 'CT_Filt_Pro_Der_SegunGen',
    'CT_Filt_Con_Izq_SegunGen', 'CT_Filt_Con_Der_SegunGen',
    'CO_Filt_Pro_Izq_SegunBal', 'CO_Filt_Pro_Der_SegunBal',
    'CO_Filt_Con_Izq_SegunBal', 'CO_Filt_Con_Der_SegunBal',
    'CT_Filt_Pro_Izq_SegunBal', 'CT_Filt_Pro_Der_SegunBal',
    'CT_Filt_Con_Izq_SegunBal', 'CT_Filt_Con_Der_SegunBal'
]

# Crear DataFrames temporales para merge
df_Gen_Merge = df_Generales[['ID'] + [v for v in Variables_Cruzadas if 'SegunBal' in v]].copy()
df_Bal_Merge = df_Ballotage[['ID'] + [v for v in Variables_Cruzadas if 'SegunGen' in v]].copy()

# Merge con df_Elecciones
# Primero, eliminar variables cruzadas si ya existen
for var in Variables_Cruzadas:
    if var in df_Elecciones.columns:
        df_Elecciones.drop(columns=[var], inplace=True)

# Merge Generales (variables SegunBal)
df_Elecciones = df_Elecciones.merge(
    df_Gen_Merge,
    on='ID',
    how='left'
)

# Merge Ballotage (variables SegunGen)
df_Elecciones = df_Elecciones.merge(
    df_Bal_Merge,
    on='ID',
    how='left'
)

print(f"✓ Variables cruzadas agregadas a df_Elecciones")
print(f"  Total columnas: {len(df_Elecciones.columns)}")
print(f"  Nuevas variables: {len(Variables_Cruzadas)}")

✓ Variables cruzadas agregadas a df_Elecciones
  Total columnas: 138
  Nuevas variables: 16


## 6. Análisis Estadístico: Comparar Variables Originales vs Cruzadas

Comparar las variables filtradas originales (usando filtro propio) vs las cruzadas.

In [8]:
print("="*70)
print("ANÁLISIS ESTADÍSTICO: FILTRO ORIGINAL vs FILTRO CRUZADO")
print("="*70)

# Verificar si existen variables originales filtradas
Variables_Originales = [
    'Cambio_Op_Filt_Pro_Izq', 'Cambio_Op_Filt_Pro_Der',
    'Cambio_Op_Filt_Con_Izq', 'Cambio_Op_Filt_Con_Der',
    'Cambio_Tiempo_Filt_Pro_Izq', 'Cambio_Tiempo_Filt_Pro_Der',
    'Cambio_Tiempo_Filt_Con_Izq', 'Cambio_Tiempo_Filt_Con_Der'
]

Comparaciones_Resultados = []

# BALLOTAGE: Comparar original (filtro Ballotage) vs cruzado (filtro Generales)
print("\n📊 BALLOTAGE: Filtro Original vs Filtro de Generales")
print("-"*70)

Mapeo_Ballotage = {
    'Cambio_Op_Filt_Pro_Izq': 'CO_Filt_Pro_Izq_SegunGen',
    'Cambio_Op_Filt_Pro_Der': 'CO_Filt_Pro_Der_SegunGen',
    'Cambio_Op_Filt_Con_Izq': 'CO_Filt_Con_Izq_SegunGen',
    'Cambio_Op_Filt_Con_Der': 'CO_Filt_Con_Der_SegunGen',
    'Cambio_Tiempo_Filt_Pro_Izq': 'CT_Filt_Pro_Izq_SegunGen',
    'Cambio_Tiempo_Filt_Pro_Der': 'CT_Filt_Pro_Der_SegunGen',
    'Cambio_Tiempo_Filt_Con_Izq': 'CT_Filt_Con_Izq_SegunGen',
    'Cambio_Tiempo_Filt_Con_Der': 'CT_Filt_Con_Der_SegunGen'
}

for var_orig, var_cruz in Mapeo_Ballotage.items():
    if var_orig in df_Ballotage.columns and var_cruz in df_Ballotage.columns:
        datos_pareados = df_Ballotage[[var_orig, var_cruz]].dropna()
        
        if len(datos_pareados) > 1:
            try:
                stat, p_valor = wilcoxon(datos_pareados[var_orig], datos_pareados[var_cruz])
                
                sig = '***' if p_valor < 0.001 else '**' if p_valor < 0.01 else '*' if p_valor < 0.05 else 'ns'
                
                print(f"\n  {var_orig}:")
                print(f"    Media Original: {datos_pareados[var_orig].mean():.4f}")
                print(f"    Media Cruzada:  {datos_pareados[var_cruz].mean():.4f}")
                print(f"    p-valor: {p_valor:.6f} ({sig})")
                
                Comparaciones_Resultados.append({
                    'Eleccion': 'Ballotage',
                    'Variable': var_orig,
                    'n': len(datos_pareados),
                    'Media_Original': datos_pareados[var_orig].mean(),
                    'Media_Cruzada': datos_pareados[var_cruz].mean(),
                    'p_valor': p_valor,
                    'Sig': sig
                })
            except Exception as e:
                print(f"\n  {var_orig}: Error - {e}")

# GENERALES: Comparar original (filtro Generales) vs cruzado (filtro Ballotage)
print("\n📊 GENERALES: Filtro Original vs Filtro de Ballotage")
print("-"*70)

Mapeo_Generales = {
    'Cambio_Op_Filt_Pro_Izq': 'CO_Filt_Pro_Izq_SegunBal',
    'Cambio_Op_Filt_Pro_Der': 'CO_Filt_Pro_Der_SegunBal',
    'Cambio_Op_Filt_Con_Izq': 'CO_Filt_Con_Izq_SegunBal',
    'Cambio_Op_Filt_Con_Der': 'CO_Filt_Con_Der_SegunBal',
    'Cambio_Tiempo_Filt_Pro_Izq': 'CT_Filt_Pro_Izq_SegunBal',
    'Cambio_Tiempo_Filt_Pro_Der': 'CT_Filt_Pro_Der_SegunBal',
    'Cambio_Tiempo_Filt_Con_Izq': 'CT_Filt_Con_Izq_SegunBal',
    'Cambio_Tiempo_Filt_Con_Der': 'CT_Filt_Con_Der_SegunBal'
}

for var_orig, var_cruz in Mapeo_Generales.items():
    if var_orig in df_Generales.columns and var_cruz in df_Generales.columns:
        datos_pareados = df_Generales[[var_orig, var_cruz]].dropna()
        
        if len(datos_pareados) > 1:
            try:
                stat, p_valor = wilcoxon(datos_pareados[var_orig], datos_pareados[var_cruz])
                
                sig = '***' if p_valor < 0.001 else '**' if p_valor < 0.01 else '*' if p_valor < 0.05 else 'ns'
                
                print(f"\n  {var_orig}:")
                print(f"    Media Original: {datos_pareados[var_orig].mean():.4f}")
                print(f"    Media Cruzada:  {datos_pareados[var_cruz].mean():.4f}")
                print(f"    p-valor: {p_valor:.6f} ({sig})")
                
                Comparaciones_Resultados.append({
                    'Eleccion': 'Generales',
                    'Variable': var_orig,
                    'n': len(datos_pareados),
                    'Media_Original': datos_pareados[var_orig].mean(),
                    'Media_Cruzada': datos_pareados[var_cruz].mean(),
                    'p_valor': p_valor,
                    'Sig': sig
                })
            except Exception as e:
                print(f"\n  {var_orig}: Error - {e}")

df_Comparaciones = pd.DataFrame(Comparaciones_Resultados)
print("\n" + "="*70)
print("✅ ANÁLISIS COMPLETADO")
print("="*70)

ANÁLISIS ESTADÍSTICO: FILTRO ORIGINAL vs FILTRO CRUZADO

📊 BALLOTAGE: Filtro Original vs Filtro de Generales
----------------------------------------------------------------------

  Cambio_Op_Filt_Pro_Izq:
    Media Original: -0.0742
    Media Cruzada:  -0.1292
    p-valor: 0.226500 (ns)

  Cambio_Op_Filt_Pro_Der:
    Media Original: -0.1168
    Media Cruzada:  0.0000
    p-valor: 0.000171 (***)

  Cambio_Op_Filt_Con_Izq:
    Media Original: 0.0829
    Media Cruzada:  0.2109
    p-valor: 0.028896 (*)

  Cambio_Op_Filt_Con_Der:
    Media Original: 0.1140
    Media Cruzada:  0.0638
    p-valor: 0.136938 (ns)

  Cambio_Tiempo_Filt_Pro_Izq:
    Media Original: 0.0000
    Media Cruzada:  -0.8603
    p-valor: 0.000000 (***)

  Cambio_Tiempo_Filt_Pro_Der:
    Media Original: 0.0000
    Media Cruzada:  -2.2430
    p-valor: 0.000000 (***)

  Cambio_Tiempo_Filt_Con_Izq:
    Media Original: -1.6320
    Media Cruzada:  -2.7781
    p-valor: 0.000001 (***)

  Cambio_Tiempo_Filt_Con_Der:
    Media O

## 7. Guardar Resultados

In [9]:
# Crear carpeta de salida si no existe
if not os.path.exists(Ruta_Procesados):
    os.makedirs(Ruta_Procesados)

# Guardar df_Elecciones actualizado
Ruta_Elecciones_Actualizado = os.path.join(Ruta_Procesados, 'df_Elecciones.xlsx')
df_Elecciones.to_excel(Ruta_Elecciones_Actualizado, index=False)
print(f"✓ df_Elecciones actualizado guardado: {Ruta_Elecciones_Actualizado}")

# Guardar resultados de comparaciones
if len(df_Comparaciones) > 0:
    Ruta_Comparaciones = os.path.join(Ruta_Procesados, 'Resultados_Filtrado_Cruzado.xlsx')
    df_Comparaciones.to_excel(Ruta_Comparaciones, index=False)
    print(f"✓ Resultados de comparaciones guardados: {Ruta_Comparaciones}")

# Guardar ítems significativos identificados
Items_Sig_Data = []
for eleccion in ['Generales', 'Ballotage']:
    for tipo in ['CO', 'CT']:
        for item in Items_Significativos[eleccion][tipo]:
            Items_Sig_Data.append({
                'Eleccion': eleccion,
                'Tipo': tipo,
                'Item': item
            })

df_Items_Sig = pd.DataFrame(Items_Sig_Data)
Ruta_Items_Sig = os.path.join(Ruta_Procesados, 'Items_Significativos_Por_Eleccion.xlsx')
df_Items_Sig.to_excel(Ruta_Items_Sig, index=False)
print(f"✓ Ítems significativos guardados: {Ruta_Items_Sig}")

print("\n✅ Todos los archivos guardados exitosamente")

✓ df_Elecciones actualizado guardado: c:\Users\Patricio\Documents\Codigo\Python\Investigacion\Tesis\Código\..\Data\Procesados\df_Elecciones.xlsx
✓ Resultados de comparaciones guardados: c:\Users\Patricio\Documents\Codigo\Python\Investigacion\Tesis\Código\..\Data\Procesados\Resultados_Filtrado_Cruzado.xlsx
✓ Ítems significativos guardados: c:\Users\Patricio\Documents\Codigo\Python\Investigacion\Tesis\Código\..\Data\Procesados\Items_Significativos_Por_Eleccion.xlsx

✅ Todos los archivos guardados exitosamente


## 8. Resumen Final

In [10]:
print("="*70)
print("RESUMEN FINAL: FILTRADO CRUZADO ENTRE ELECCIONES")
print("="*70)

print("\n📊 Variables creadas (16 en total):")
print("\n  En Ballotage (usando filtro de Generales):")
print("    - 4 variables CO: Pro_Izq, Pro_Der, Con_Izq, Con_Der")
print("    - 4 variables CT: Pro_Izq, Pro_Der, Con_Izq, Con_Der")
print("\n  En Generales (usando filtro de Ballotage):")
print("    - 4 variables CO: Pro_Izq, Pro_Der, Con_Izq, Con_Der")
print("    - 4 variables CT: Pro_Izq, Pro_Der, Con_Izq, Con_Der")

print("\n📈 Análisis realizados:")
print("  1. Kruskal-Wallis para identificar ítems significativos")
print("  2. Creación de variables filtradas cruzadas")
print("  3. Wilcoxon pareado: Filtro Original vs Filtro Cruzado")

print("\n📁 Archivos generados:")
print("  - df_Elecciones.xlsx (actualizado con 16 nuevas variables)")
print("  - Resultados_Filtrado_Cruzado.xlsx (comparaciones estadísticas)")
print("  - Items_Significativos_Por_Eleccion.xlsx (listado de ítems)")

print("\n🎯 Interpretación:")
print("  - Si Filtro Original ≈ Filtro Cruzado → Hallazgos robustos entre elecciones")
print("  - Si Filtro Original ≠ Filtro Cruzado → Ítems específicos de cada elección")
print("  - Permite validación cruzada de resultados")

if len(df_Comparaciones) > 0:
    sig_count = len(df_Comparaciones[df_Comparaciones['Sig'] != 'ns'])
    print(f"\n📊 Resultados:")
    print(f"  - Total comparaciones: {len(df_Comparaciones)}")
    print(f"  - Diferencias significativas: {sig_count}")
    print(f"  - No significativas: {len(df_Comparaciones) - sig_count}")

print("\n" + "="*70)
print("✅ ANÁLISIS COMPLETADO")
print("="*70)

RESUMEN FINAL: FILTRADO CRUZADO ENTRE ELECCIONES

📊 Variables creadas (16 en total):

  En Ballotage (usando filtro de Generales):
    - 4 variables CO: Pro_Izq, Pro_Der, Con_Izq, Con_Der
    - 4 variables CT: Pro_Izq, Pro_Der, Con_Izq, Con_Der

  En Generales (usando filtro de Ballotage):
    - 4 variables CO: Pro_Izq, Pro_Der, Con_Izq, Con_Der
    - 4 variables CT: Pro_Izq, Pro_Der, Con_Izq, Con_Der

📈 Análisis realizados:
  1. Kruskal-Wallis para identificar ítems significativos
  2. Creación de variables filtradas cruzadas
  3. Wilcoxon pareado: Filtro Original vs Filtro Cruzado

📁 Archivos generados:
  - df_Elecciones.xlsx (actualizado con 16 nuevas variables)
  - Resultados_Filtrado_Cruzado.xlsx (comparaciones estadísticas)
  - Items_Significativos_Por_Eleccion.xlsx (listado de ítems)

🎯 Interpretación:
  - Si Filtro Original ≈ Filtro Cruzado → Hallazgos robustos entre elecciones
  - Si Filtro Original ≠ Filtro Cruzado → Ítems específicos de cada elección
  - Permite validación c